In [ ]:
##TO Make video showing key years of ualong propagation

In [1]:
## check key times for ASC velocity increase 0 
## annual average movie


In [1]:
import cosima_cookbook as cc
from dask.distributed import Client
import matplotlib.pyplot as plt
import matplotlib.path as mpath
import numpy as np

import xarray as xr
import cmocean as cm
import cartopy.crs as ccrs
import cmocean as cm
import glob
import os

In [2]:
import matplotlib.dates as mdates
import datetime
import pandas as pd

In [3]:
%matplotlib inline

In [4]:
from matplotlib import rc
rc('font', **{'size':25})
rc('text', usetex=False)
rc('xtick', labelsize=25) #20 
rc('ytick', labelsize=25) #20
rc('axes', titlesize=25)    # fontsize of the axes title
rc('axes', labelsize=25) #24    # fontsize of the x and y labels

In [5]:
session=cc.database.create_session()
expt = '01deg_jra55v13_ryf9091'

In [6]:
import dask.config
from dask.distributed import Client,LocalCluster
from dask_jobqueue import PBSCluster
'''
walltime = '03:00:00'
cores = 28

memory = '252GB'

cluster = PBSCluster(walltime=str(walltime), cores=cores, memory=str(memory),processes=cores,
                     job_extra=['-q normalbw','-P v45','-l ncpus='+str(cores),'-l mem='+str(memory),
                                '-l storage=gdata/ik11+gdata/e14+gdata/hh5+gdata/v45+gdata/cj50',
                                '-l jobfs=400GB'],
                     local_directory='$TMPDIR',
                     header_skip=["select"],
                     #python=os.environ["DASK_PYTHON"]
                    )
cluster

# and if you want even more power, you can use this to make it three nodes
cluster.scale(jobs=3)
'''
# then, run this to make a dask client

c = Client()
c

Connection method: Cluster object,Cluster type: distributed.LocalCluster
Dashboard: /proxy/36691/status,
Dashboard: /proxy/36691/status,Workers: 2
Total threads: 2,Total memory: 9.00 GiB
Status: running,Using processes: True
Comm: tcp://127.0.0.1:33527,Workers: 2
Dashboard: /proxy/36691/status,Total threads: 2
Started: Just now,Total memory: 9.00 GiB
Comm: tcp://127.0.0.1:46215,Total threads: 1
Dashboard: /proxy/36737/status,Memory: 4.50 GiB
Nanny: tcp://127.0.0.1:42079,


In [7]:
hu = cc.querying.getvar(expt='01deg_jra55v13_ryf9091', variable='hu', 
                          session=session, frequency='static',
                          attrs={'cell_methods': 'time: point'}, n=1).sel(yu_ocean = slice(-79,-60)).chunk({ 'yu_ocean':69, 'xu_ocean':360})
hu

<xarray.DataArray 'hu' (yu_ocean: 440, xu_ocean: 3600)> Size: 6MB
dask.array<rechunk-merge, shape=(440, 3600), dtype=float32, chunksize=(69, 360), chunktype=numpy.ndarray>
Coordinates:
  * xu_ocean  (xu_ocean) float64 29kB -279.9 -279.8 -279.7 ... 79.8 79.9 80.0
  * yu_ocean  (yu_ocean) float64 4kB -78.98 -78.93 -78.89 ... -60.07 -60.02
    geolon_c  (yu_ocean, xu_ocean) float32 6MB dask.array<chunksize=(69, 360), meta=np.ndarray>
    geolat_c  (yu_ocean, xu_ocean) float32 6MB dask.array<chunksize=(69, 360), meta=np.ndarray>
Attributes:
    long_name:     ocean depth on u-cells
    units:         m
    valid_range:   [-1.e+09  1.e+09]
    cell_methods:  time: point
    ncfiles:       ['/g/data/ik11/outputs/access-om2-01/01deg_jra55v13_ryf909...
    contact:       Andy Hogg
    email:         andy.hogg@anu.edu.au
    created:       2020-06-11
    description:   0.1 degree ACCESS-OM2 global model configuration with JRA5...
    notes:         Additional daily outputs saved from 1 Jan 1950 to 31 Dec 1...
    url:           https://github.com/COSIMA/01deg_jra55_ryf/tree/01deg_jra55...

In [8]:
# location limits of dataset - choosing slice near totten
#lon_lim = slice(-270, -230) #slice(30, 45)#
lat_lim = slice(-79, -60) #slice(-70, -65)

hu_region = hu.sel( yu_ocean = lat_lim)

In [9]:

exp_ctrl= '01deg_jra55v13_ryf9091'
exp_wthmp= '01deg_jra55v13_ryf9091_qian_wthmp'
exp_wthp= '01deg_jra55v13_ryf9091_qian_wthp'
lat_lim = slice(-79, -60)
lon_lim = slice(-279.99,80)

#### SET EXPT HERE
expt = exp_wthp


In [10]:
## plotting everything together
cluster_mask_access = xr.open_mfdataset('Antarctic-Eddy-Res-Compare/largest_3cluster_mask_xr_accessom201.nc')
cluster_mask_access= cluster_mask_access.assign_coords(xu_ocean = ('xu_ocean', hu.xu_ocean.data), \
                                                       yu_ocean = ('yu_ocean', hu.yu_ocean.data))


In [11]:
# We will use an annual average from 10 year in the RYF run
start_time_mw='2110-01-31 00:00:00' 
end_time_mw='2159-12-31 00:00:00'


In [12]:
land_mask = np.squeeze(hu.values)
land_mask = land_mask * 0
land_mask[np.isnan(land_mask)] = 1
yu_ocean = hu.yu_ocean.values
xu_ocean = hu.xu_ocean.values
land_mask_masked = np.ma.masked_where((land_mask==0),land_mask)
# make land go all the way to -90S:
land_mask_lat = hu.yu_ocean.values
land_mask_lat[0] = -80


In [13]:
def changeline_cs(linestyle):
    cs.monochrome = True
    for col,  ls in zip(cs.collections,cs._process_linestyles(linestyle)):
        col.set_linestyle(ls)
        col.set_edgecolor('k')
        #col.set_linewidth(2)


In [14]:
import cmocean
from cartopy.mpl.ticker import LongitudeFormatter, LatitudeFormatter

In [15]:
FileList = ['/g/data/v45/qo9901/MW-perturb-ASC-data/ualongdepth_annualmean_' + exp_wthmp + f'_{tt:03d}.nc' for tt in range(50)]
u_alongdepth_arr_wthmp = xr.open_mfdataset(FileList, combine = 'by_coords')
FileList = ['/g/data/v45/qo9901/MW-perturb-ASC-data/ualongdepth_annualmean_' + exp_wthp + f'_{tt:03d}.nc' for tt in range(50)]
u_alongdepth_arr_wthp = xr.open_mfdataset(FileList, combine = 'by_coords')
u_alongdepth_arr_wthmp

<xarray.Dataset> Size: 24GB
Dimensions:       (st_ocean: 75, xu_ocean: 3600, yu_ocean: 440, year: 50)
Coordinates:
  * st_ocean      (st_ocean) float64 600B 0.5413 1.681 ... 5.511e+03 5.709e+03
  * xu_ocean      (xu_ocean) float64 29kB -279.9 -279.8 -279.7 ... 79.9 80.0
  * yu_ocean      (yu_ocean) float64 4kB -78.98 -78.93 -78.89 ... -60.07 -60.02
  * year          (year) int64 400B 0 1 2 3 4 5 6 7 ... 42 43 44 45 46 47 48 49
Data variables:
    ualong_depth  (year, st_ocean, yu_ocean, xu_ocean) float32 24GB dask.array<chunksize=(1, 75, 440, 3600), meta=np.ndarray>

In [16]:
# Import edges of st_ocean and add lat/lon dimensions:  , start_time=start_time_mw, end_time=end_time_mw,
st_edges_ocean = cc.querying.getvar(expt, 'st_edges_ocean', session, n=1)
st_edges_array = st_edges_ocean.expand_dims({'yu_ocean': hu.yu_ocean, \
                                             'xu_ocean':hu.xu_ocean}, axis=[1, 2])

# Adjust edges at bottom for partial thickness:
st_edges_with_partial = st_edges_array.where(st_edges_array<hu, other=hu)
thickness = st_edges_with_partial.diff(dim='st_edges_ocean')

# Change coordinate of thickness to st_ocean (needed for multipling with other variables):
st_ocean = cc.querying.getvar(expt, 'st_ocean', session, n=1)
thickness['st_edges_ocean'] = st_ocean.values
thickness = thickness.rename(({'st_edges_ocean': 'st_ocean'}))
thickness = thickness.sel(st_ocean=slice(0, 500))

#v_barotropic = (v_cross * thickness).sum('st_ocean') / thickness.sum('st_ocean')

In [17]:
def ualong_barotropic(u_alongdepth_arr, yr):
    # Depth average gives us the barotropic velocity
    u_barotropic = (u_alongdepth_arr.ualong_depth.isel(year = yr) * thickness).sum('st_ocean') / thickness.sum('st_ocean')
    return u_barotropic

In [18]:
levels_ualong = np.arange(-0.085,0.085,0.005)

In [ ]:
for tt in range(21,50):
    fig, ax = plt.subplots(figsize = (20,8))
    
    ualong_mw_barotropic = ualong_barotropic(u_alongdepth_arr_wthmp,tt) - ualong_barotropic(u_alongdepth_arr_wthp,tt)
    
    # accessom2-01
    cbar_kwargs = {'location':'horizontal', 'label': 'ualong (m/s)'}
    cmap = ualong_mw_barotropic.where(cluster_mask_access == True).__xarray_dataarray_variable__\
                .plot.contourf(levels = levels_ualong, cbar_kwargs = dict({'location':'right', 'label': r'u$_{along}$ (m/s)'}))
    #cs = hu_region.where(hu_region < 2600).plot.contour(levels  = [2500], color = 'k', ax = ax[0])
    #changeline_cs('solid')
    cs = hu_region.plot.contour(levels  = [1000], color = 'k', linestyle = 'dashed') #conventional definition for ASC
    changeline_cs('dashed')
    ax.set_ylabel('')
    ax.contourf(hu.xu_ocean, land_mask_lat, land_mask_masked, colors='darkgrey', zorder=2)
    ax.text(0.01,0.05,'Meltwater \nComponent Year = '+ str(tt),transform=ax.transAxes)
    ax.set_xlabel('')
    
    '''
    cb_ax = fig.add_axes([0.1, -0.01, 0.8, 0.02])
    cbar = plt.colorbar(cmap, label = '$u_{along}$ (m/s)', orientation = 'horizontal', cax = cb_ax, pad = 0.01)
    cbar.set_ticks(levels_ualong[::4])
    '''
    ax.yaxis.set_major_formatter(LatitudeFormatter())
    ax.xaxis.set_major_formatter(LongitudeFormatter())
    plt.tight_layout()
    fig.savefig(f'/g/data/v45/qo9901/MW-perturb-ASC-data/movie_MW_component_propagation/MW_ualongbarotropic_{tt:03d}.png', dpi= 600)
    print(tt)

/g/data/hh5/public/apps/miniconda3/envs/analysis3-24.01/lib/python3.10/site-packages/dask/core.py:127: RuntimeWarning: invalid value encountered in divide
  return func(*(_execute_task(a, cache) for a in args))
/g/data/hh5/public/apps/miniconda3/envs/analysis3-24.01/lib/python3.10/site-packages/dask/core.py:127: RuntimeWarning: invalid value encountered in divide
  return func(*(_execute_task(a, cache) for a in args))


21
22
23
24
25
26
27
28
